# How LLMs Work 
This notebook will provide an overview of how language models (LLMs) work. We'll cover the key components and processes involved in generating text, including encoding, decoding, and attention mechanisms.

@image url="" alt="Example Image"

In [1]:
!pip uninstall -y numpy matplotlib
!pip install numpy
!pip install matplotlib --force-reinstall
!pip install  ipywidgets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 22.6 MB/s eta 0:00:00 MB/s eta 0:00:0101
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 16.9 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.5 MB/s eta 0:00:00
Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux201

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
from IPython.display import display

# Set random seed for reproducibility
np.random.seed(42)

# Create a function to generate the plot
def create_regression_plot(slope=1.0, intercept=0.5, noise=0.7, num_points=40):
    """
    Create a linear regression visualization with customizable line parameters.
    
    Parameters:
    -----------
    slope : float
        The slope of the line (default: 1.0)
    intercept : float
        The y-intercept of the line (default: 0.5)
    noise : float
        The amount of noise to add to the points (default: 0.7)
    num_points : int
        Number of datpip uninstall numpy
a points to generate (default: 40)
    """
    # Create figure with black background
    plt.figure(figsize=(12, 8), facecolor='black')
    ax = plt.subplot(111)
    ax.set_facecolor('black')
    
    # Generate x values (square footage)
    x = np.random.uniform(0.2, 10, num_points)
    
    # Generate y values with the specified line function and noise
    y = slope * x + intercept + np.random.normal(0, noise, num_points)
    
    # Create the plot
    ax.scatter(x, y, color='#FFD700', s=50, alpha=0.8)
    
    # Create x values for the line
    line_x = np.array([0, 12])
    
    # Calculate y values for the line using the current slope and intercept
    line_y = slope * line_x + intercept
    
    # Plot the line
    ax.plot(line_x, line_y, color='#90EE90', linewidth=2)
    
    # Set axis colors to white
    ax.spines['bottom'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Set tick colors to white
    ax.tick_params(axis='x', colors='white', which='both')
    ax.tick_params(axis='y', colors='white', which='both')
    
    # Set axis labels with larger font
    ax.set_xlabel('Square footage', color='white', fontsize=20)
    ax.set_ylabel('Output', color='white', fontsize=20)
    
    # Set title
    ax.set_title('Linear Regression', color='white', fontsize=30)
    
    # Set axis limits
    ax.set_xlim(0, 12)
    ax.set_ylim(0, 13)
    
    # Show gridlines
    ax.grid(True, alpha=0.2, color='white')
    
    plt.tight_layout()
    plt.show()
    
    # Print the line equation
    print(f"Line equation: y = {slope:.2f}x + {intercept:.2f}")

# Create interactive widgets
slope_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=3.0,
    step=0.1,
    description='Slope:',
    continuous_update=False
)

intercept_slider = widgets.FloatSlider(
    value=0.5,
    min=-2.0,
    max=5.0,
    step=0.1,
    description='Intercept:',
    continuous_update=False
)

noise_slider = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Noise:',
    continuous_update=False
)

points_slider = widgets.IntSlider(
    value=40,
    min=10,
    max=100,
    step=5,
    description='Points:',
    continuous_update=False
)

# Create interactive output
interactive_plot = widgets.interactive_output(
    create_regression_plot,
    {'slope': slope_slider, 'intercept': intercept_slider, 'noise': noise_slider, 'num_points': points_slider}
)

# Display the widgets and plot
controls = widgets.VBox([slope_slider, intercept_slider, noise_slider, points_slider])
output = widgets.VBox([interactive_plot])
display(widgets.HBox([controls, output]))

# Initial plot
create_regression_plot()